In [ ]:
# import arcpy
import os
import numpy
import pandas
from datetime import datetime, timedelta

In [ ]:
arcpy.env.workspace = "C:\\Users\\Jayant\\Desktop\\SSTD\\workspace"
print(arcpy.env.workspace)

in_filename = "C:\\Users\\Jayant\\Desktop\\SSTD\\outputs\\rti_joined_race_income_hhsize_features.csv"
def getCountyIoD(in_filename, level_of_aggregation):
    rows = pandas.read_csv(filepath_or_buffer=os.path.join(arcpy.env.workspace, in_filename),
                                 header=0,
                                 index_col=False,
                                 usecols=['sp_hh_id', 'hh_race', 'hh_size', 'GEOID20', "COUNTYFP20", 
                                          "TRACTCE20", "GEOID20_1"],
                                 engine='c')
    return rows
    
rows = getCountyIoD(in_filename, None)

In [ ]:
# Function to update header, remove erroneous rows, update col. type and filter the dataset
def preprocessing(rows, dropColumns):
    # Rename headers
    rows = rows.rename(columns={'GEOID20':'BLOCK_ID', 'GEOID20_1':'BLCKGRP_ID'})
    rows = rows[rows['COUNTYFP20'] != ' '] # Remove rows with no county
    rows['COUNTYFP20'] = rows['COUNTYFP20'].astype('int64') # Update column data type.
    filter_data = rows.drop(columns=dropColumns) # Subsetting the data
    print(filter_data.head()) # Print to check the results.
    return filter_data

# Function to group the dataset
def grouping(filter_data, groupByArg):
    grouped_df = filter_data.groupby(groupByArg).agg({'hh_race':list, 'hh_size':list}) # Group by 
    grouped_df = grouped_df.reset_index()
    print(grouped_df.head())
    return grouped_df
    
# tract_data = preprocessing(rows, ['BLOCK_ID', 'BLCKGRP_ID'])
# grouped_tract_df = grouping(tract_data, ['COUNTYFP20','TRACTCE20'])

# block_grp_data = preprocessing(rows, ['TRACTCE20', 'BLOCK_ID'])
# grouped_bg_df = grouping(block_grp_data, ['COUNTYFP20','BLCKGRP_ID'])

block_data = preprocessing(rows, ['TRACTCE20', 'BLCKGRP_ID'])
grouped_block_df = grouping(block_data, ['COUNTYFP20','BLOCK_ID'])

In [ ]:
# Function to create distribution for each county.
def CreateDIST(grouped_df, agg_level):
    data_store = {}
    counties = grouped_df['COUNTYFP20'].unique()
    test_flag = 1
    for county_id in counties:
        print("Currently processing county #{0}".format(county_id))
        tract_store = {}
        county_data = grouped_df[grouped_df['COUNTYFP20'] == county_id] # Get the data for each county.
  
        for index, row in county_data.iterrows(): # Iterate over each tract.
            #print(row)
            tract_race_dist = {}
            tract_id = row[agg_level] # <-- agg_level argument is here.
            n_household = len(row['hh_race'])
            if not (len('hh_race') == len('hh_size')):
                print("Sanity check failed")
            print("There are {0} households in block #{1}".format(n_household, tract_id))
            list_race = row['hh_race']
            list_size = row['hh_size']
            for i in range(0, n_household): # Iterate over each household in every tract.
                key = list_race[i] # Get the race of the current household.
                if key in tract_race_dist:
                    tract_race_dist[key] += int(list_size[i]) # Add the size of each household.
                else:
                    tract_race_dist[key] = int(list_size[i])
            tract_store[tract_id] = tract_race_dist
        data_store[county_id] = tract_store
    return data_store
#    if(test_flag == 1):
#        break
#data_store = CreateDIST(grouped_df, 'TRACTCE20')
#block_grp_data_store = CreateDIST(grouped_bg_df, 'BLCKGRP_ID')
block_data_store = CreateDIST(grouped_block_df, 'BLOCK_ID')
print("Datastore created")

In [ ]:
print(len(data_store))
print(len(block_grp_data_store))
print(len(block_data_store))
#keys = list(data_store.keys())
print(data_store[1])
print(block_grp_data_store[1])
print(block_data_store[1])

In [ ]:
def createCountyDIST(data_store):
    racial_dist = {}
    for County in data_store:
        Tracts = data_store[County].keys()
        dist = {}
        for race in range(1, 10):
            dist[race] = []
    #    print(dist)
        for Tract in Tracts: # Compute the racial distribution of counties grouped by census tracts.
            for race in range(1, 10):
                if race in data_store[County][Tract]:
                    dist[race].append(data_store[County][Tract][race])
                else:
                    dist[race].append(0)
        racial_dist[County] = dist
    return racial_dist
tract_racial_dist = createCountyDIST(data_store)
print(tract_racial_dist[1])
block_grp_racial_dist = createCountyDIST(block_grp_data_store)
print(block_grp_racial_dist[1])
block_racial_dist = createCountyDIST(block_data_store)
print(block_racial_dist[1])

In [ ]:
def createStateDIST(tract_racial_dist):
    county_racial_dist = {}
    for race in range(1, 10):
        county_racial_dist[race] = []
    for county in tract_racial_dist.keys():
        for race in range(1,10):
            total = sum(tract_racial_dist[county][race])
            county_racial_dist[race].append(total)
    return county_racial_dist
county_racial_dist = createStateDIST(tract_racial_dist)
print(county_racial_dist)
print(len(county_racial_dist[1]))

In [ ]:
def computeMEI(Agg_DIST, Part_DIST, COUNTY_INDEX):
    T = 0
    for key in Agg_DIST.keys():
        T += sum(Agg_DIST[key])
    
    N_Tracts = len(Part_DIST[1])
    Ti = [0 for i in range(0, N_Tracts)]
    for i in range(0, N_Tracts):
        for key in Part_DIST.keys():
            Ti[i] += Part_DIST[key][i]
    
    E = 0.0
    for key in Agg_DIST.keys():
        E += float(Agg_DIST[key][COUNTY_INDEX]) / sum(Agg_DIST[key])
    
    Ei = [0 for i in range(0, N_Tracts)]
    for i in range(0, N_Tracts):
        for key in Part_DIST.keys():
#             print(Part_DIST[key])
#             print(N_Tracts)
#             print(Part_DIST[key][i])
            Ei[i] += float(Part_DIST[key][i]) / (sum(Part_DIST[key]) + 0.0001)
    
    MEI = 0.0
    for i in range(0, N_Tracts):
        MEI += (float(Ti[i])/T) * ((E - Ei[i]) / E)
    return MEI

def countyMultivariateMEIRanker(Agg_DIST, racial_dist):
    County_MEI = {}
    county_index = 0
    for key in racial_dist.keys():
        #print(key)
        MEI = computeMEI(Agg_DIST, racial_dist[key], county_index)
        County_MEI[key] = MEI
        county_index += 1
    ranked_MEI = dict(sorted(County_MEI.items(), key=lambda item: item[1], reverse=True))
    print(ranked_MEI)
    return ranked_MEI
#print(tract_racial_dist.keys())
ranked_MEI = countyMultivariateMEIRanker(county_racial_dist, tract_racial_dist)

In [ ]:
# Function to coumpute IOD for two different distributions.
def computeIOD(l1, l2):
    l1_sum = sum(l1)
    l2_sum = sum(l2)
    if (l1_sum == 0 or l2_sum == 0):
        return 0.0
    
    norm_l1 = [float(i)/l1_sum for i in l1]
    norm_l2 = [float(i)/l1_sum for i in l2]
    
    #print("l1={0} \nl2={1} \nl1-norm={2} \nl2-norm={3}".format(l1, l2, norm_l1, norm_l2))
    
    list_diff = [norm_l1[i] - norm_l2[i] for i in range(len(norm_l1))]
    abs_sum = sum([abs(number) for number in list_diff])
   # print("Index of Dissimilarity = {0}".format(abs_sum/2))
    return abs_sum / 2

# Currently merging race 1 vs others.
from operator import add
def countyBivariateIODRanker(racial_dist):
    # Compute IOD for each County
    County_IOD = {} # Storing the Index of Dissimiliarity for a county.
    for key in racial_dist:
        init_list = racial_dist[key][2]
        for i in range(3, 10):
            init_list = list(map(add, init_list, racial_dist[key][i]))
        IOD = computeIOD(racial_dist[key][1], init_list)
        County_IOD[key] = IOD
    ranked_County = dict(sorted(County_IOD.items(), key=lambda item: item[1], reverse=True))
    print(ranked_County)
    return ranked_County

tract_ranked_County = countyBivariateIODRanker(tract_racial_dist)
block_grp_ranked_County = countyBivariateIODRanker(block_grp_racial_dist)
block_ranked_County = countyBivariateIODRanker(block_racial_dist)

In [ ]:
# Function to add names to county ids and provide a ranked (descending) list of counties.
def countyNamedList(ranked_County):
    #Create dataframe for IOD.
    County_IOD = pandas.DataFrame(ranked_County.items(), columns = ['COUNTYFP20', 'IOD'])
    # Index the df on COUNTY IDs.
    County_IOD.set_index('COUNTYFP20')

    # Read County shapefile.
    arr = arcpy.da.TableToNumPyArray('tl_2020_27_county20.shp', ('COUNTYFP20', 'NAMELSAD20'))
    # Create dataframe for county names.
    County_Name = pandas.DataFrame(arr)
    County_Name['COUNTYFP20'] = County_Name['COUNTYFP20'].astype('int64')
    County_Name.set_index('COUNTYFP20')

    # Create the ranked list for all the counties.
    Ranked_List = County_Name.merge(County_IOD)

    # Sort counties in descending order of their IOD.
    Ranked_List = Ranked_List.sort_values(by = ['IOD'], ascending=False)
    return Ranked_List
tract_Ranked_List = countyNamedList(tract_ranked_County)
block_grp_Ranked_List = countyNamedList(block_grp_ranked_County)
block_Ranked_List = countyNamedList(block_ranked_County)
tract_Ranked_MEI_List = countyNamedList(ranked_MEI)

In [ ]:
pandas.set_option('display.max_rows', 10)
print(tract_Ranked_List)
print(block_grp_Ranked_List)
print(block_Ranked_List)
print(tract_Ranked_MEI_List)

In [ ]:
Counties = list(tract_Ranked_List['NAMELSAD20'])
Rankings = dict.fromkeys(Counties)
for key in Rankings.keys():
    Rankings[key] = [0,0,0]
for i in range(0, len(tract_Ranked_List)):
    tract_row = tract_Ranked_List.iloc[i]
    block_grp_row = block_grp_Ranked_List.iloc[i]
    block_row = block_Ranked_List.iloc[i]
    
    Rankings[tract_row['NAMELSAD20']][0] = i+1
    Rankings[block_grp_row['NAMELSAD20']][1] = i+1
    Rankings[block_row['NAMELSAD20']][2] = i+1
#    print("{0} - {1} - {2}".format(, block_grp_row['NAMELSAD20'], block_row['NAMELSAD20']))    
for key in Rankings.keys():
    print("{0}{1}".format(key, Rankings[key]))